In [1]:
from signals import complex_signal, buy_sell_signal, diff_signal
from datasets.kaggle import KaggleDataset
import pandas as pd
import numpy as np

In [2]:

def get_price_date(start: pd.Timestamp, end: pd.Timestamp) -> tuple[np.ndarray, np.ndarray]: # -> price, date
    dataset = KaggleDataset("Daily")
    df = dataset.get_timerange(start, end)
    return df["close"].values, df["date"].values

price, date = get_price_date(pd.Timestamp("2018-1-1"), pd.Timestamp("2020-1-1"))

In [3]:
def eval_fn(params):
    hw1, hw2, hw3, hd1, hd2, hd3, hsf, lw1, lw2, lw3, ld1, ld2, ld3, lsf = params

    high_freq = complex_signal(price, hw1, hw2, hw3, hd1, hd2, hd3, hsf)
    low_freq = complex_signal(price, lw1, lw2, lw3, ld1, ld2, ld3, lsf)

    buy_sell = buy_sell_signal(diff_signal(high_freq, low_freq))

    # assuming each trade will trade all units and each trade has a fee of 3% calculate the cash held at the end given 1000 initial cash
    cash = 1000
    bitcoin = 0
    for p, bs in zip(price, buy_sell):
        if bs > 0:
            bitcoin = (cash * (1 - 0.03)) / p
            cash = 0
        elif bs < 0:
            cash = (bitcoin * (1 - 0.03)) * p
            bitcoin = 0

    if bitcoin > 0:
        cash = (bitcoin * (1 - 0.03)) * price[-1]
        bitcoin = 0

    return cash

In [9]:
# Test eval # EMA is high freq, LMA is low freq

W = 100
eval_fn([0, 0, 1, 0, 0, W, 2/(W+1), 0, 1, 0, 0, W, 0, 0])

np.float64(2349.3667665590106)